In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import gc
gc.collect()

import cv2
import time
import random
import natsort
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["axes.grid"]=False

# Import pytorch modules
import torch
torch.cuda.empty_cache()

import dlib
from imutils import face_utils
import torchvision.transforms.functional as TF
from torchvision import models
# Import local modules
from src.models.nnabla_model import *
from src.utils.get_list import *
from src.utils.print_overwrite import *

In [ ]:
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'

SRC_ROOT = "/home/ubuntu/workspace/FLD-scratch/src/"

INPUT_ROOT = "/data/komedi/k-face-100"
IMAGE_ROOT = os.path.join(INPUT_ROOT, "cropped_img")
IMAGE_LIST = get_file_list(IMAGE_ROOT, case="image")

# pretrained_model = "/home/ubuntu/workspace/FLD-scratch/src/pretrained_model/mobilenet_224_model_best_gdconv_external.pth.tar"
# model_name = Path(pretrained_model).stem
model = resnet_depth



